# Normal mode viewing tool #

The purpose of this script is to take the variables file and the excel spreadsheet (Molecule_Normal_modes) produced by the "Pipeline for Gaussian results" script (Pipeline_Gaussian) and extract high contributing normal mode displacements (i.e. the molecular movements expressed as bond stretching/bending/wagging that are most intensive in a vibrational mode) for a selection of normal modes given by the user. 

In this work, bond "stretching" is defined as a change in bond length (R) between 2 atoms, bond "bending" is defined as a change in bond angle (A) between 3 atoms and bond "wagging" is defined as a change in dihedral bond angle (D) between 4 atoms.

"Combined" contribution modes in this script refer to two displacements that show the same atomic "movement" (e.g. if we imagine a hydrogen atom on the ortho-position of a benzene ring (Carbons named C1, C2 ... C6 clock-wise), performing an in-plane bending vibration, the same vibration is described by the change between both the H-C2-C1 and H-C2-C3 bond angles) and so if both of those changes add up to a significant contribution, they are are also included. 

Unlike other scripts, this script is intended to be used as a functional tool by itself rather than just produce a flow of data. The user can select a molecule and a list of normal modes (as they are indexed in the Gaussian results file) and the script will produce a dictionary of dictionaries of all displacements with their contributions for all vibrational modes (i.e. a labelled 2-column "table" in which each row represents each selected vibrational mode, giving access to another storage order of each vibrational mode's "Stretching", "Bending" and "Wagging" vibrations as a table with each important displacement (>4% selected here) and its percentage contribution).

This works quite well using the Spyder Python environment, as it provides a "variable explorer" GUI, on which the produced tables can simply be clicked on and opened (see below).

![alt text](Pipeline_modes_1.jpg "Produced variable")

The "Vibrations_vanillin" variable contains the organised information (name of the molecule replaces vanillin for other molecules).

![alt text](Pipeline_modes_2.jpg "Dictionary of dictionaries")

Clicking on it opens up the dictionary containing all vibrational modes.

![alt text](Pipeline_modes_3.jpg "Normal mode")

Clicking on a vibrational mode (in this case number 20) opens the stored information on important stretching, bending and wagging bond displacements.

![alt text](Pipeline_modes_4.jpg "Important vibrations")

Clicking on one category of displacements (in this case "Wagging" displacements).

Importing libraries.

In [1]:
import pandas as pd
import numpy as np
import json
from pandas import ExcelWriter
import matplotlib.pyplot as plt

This is the section in which the molecule name (all lower-case, keep in mind that the variables file can alter names to make them readable in code (replacing characters like ",", "+", "-"), a list with all requested modes and a path to the spreadsheet produced by the Pipeline script. Threshold refers to the "percentage" point above which displacements should be included.

In [2]:
Molecule = "vanillin"

Modes = [20, 21, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 39, 40, 41]

path = Molecule + "_normal_modes.xlsx"

Threshold = 4

This section goes through the normal modes of the spreadsheet, extracts the appropriate displacements (those that meet the threshold criteria) and then organises them into the final dictionary database.

In [3]:
exec("Vibrations_" + Molecule + "= {}")
for mode in Modes:   
    Vibrations = {}
    df = pd.read_excel(path, sheet_name = "Normal_mode_"+str(mode))
    df = df.apply(lambda x: pd.Series(x.dropna().values)).fillna('')
        
    single_stretching_vibrations = {}
    single_bending_vibrations = {}
    single_wagging_vibrations = {}
    
    combined_bending_vibrations = {}
    combined_wagging_vibrations = {}
    
    counter = 0
    for i in df.iloc[:,2]:
        if i != "" and float(i) > Threshold:
            single_stretching_vibrations[str(df.iloc[counter, 0])]= i
        counter = counter + 1
         
    def common_atoms(a, b):
        return list(np.intersect1d(a, b))
    
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    
    counter_i = -1
    
    for i in df.iloc[:,3]:
        if i != "":
            counter_j = -1
            counter_i = counter_i + 1
            i = i.replace("(", "[")
            i = i.replace(")", "]")
            i = json.loads(i)
            for j in df.iloc[:,3]:
                if j != "":
                    counter_j = counter_j + 1
                    j = j.replace("(", "[")
                    j = j.replace(")", "]")
                    j = json.loads(j)
                    if j != i and j != "":
                        if df.iloc[counter_j, 5] > Threshold:
                            single_bending_vibrations[str(j)]= float(df.iloc[counter_j,5])
                        if i[1] == j[1] and i[2] == j[2] and float(df.iloc[counter_i,5]) > Threshold / 2 and float(df.iloc[counter_j,5]) > Threshold / 2 and float(df.iloc[counter_i,5])+float(df.iloc[counter_j,5]) > Threshold:
                            combined_bending_vibrations[str(i) + "," + str(j)] = float(df.iloc[counter_i,5])+float(df.iloc[counter_j,5])
                        elif i[0] == j[0] and i[1] == j[1] and float(df.iloc[counter_i,5]) > Threshold / 2 and float(df.iloc[counter_j,5]) > Threshold / 2 and float(df.iloc[counter_i,5])+float(df.iloc[counter_j,5]) > Threshold:
                            combined_bending_vibrations[str(i) + "," + str(j)] = float(df.iloc[counter_i,5])+float(df.iloc[counter_j,5])
    
            
    
    
    lst_to_pop= []
    lst_counted = []
    for i in combined_bending_vibrations.keys():
        if i not in lst_counted:
            i_new = "["+i+"]"
            i_new = json.loads(i_new)
            for j in combined_bending_vibrations.keys():
                j_new = "["+j+"]"
                j_new = json.loads(j_new)
                if j_new[0] == i_new[1] and i_new[0] == j_new[1] and i != lst_counted:
                        lst_to_pop.append(j)
                        lst_counted.append(j)
                    
    for i in lst_to_pop:
        combined_bending_vibrations.pop(i)
        
    combined_bending_vibrations.update(single_bending_vibrations)
    
    counter_i = -1
    
    for i in df.iloc[:,6]:
        if i != "":
            counter_j = -1
            counter_i = counter_i + 1
            i = i.replace("(", "[")
            i = i.replace(")", "]")
            i = json.loads(i)
            for j in df.iloc[:,6]:
                counter_j = counter_j + 1
                if j != i and j != "":
                    j = j.replace("(", "[")
                    j = j.replace(")", "]")
                    j = json.loads(j)
                    if df.iloc[counter_j, 8] > Threshold:
                        single_wagging_vibrations[str(j)]= float(df.iloc[counter_j,8])
                    if len(intersection(i, j)) == 3 and float(df.iloc[counter_i,8]) > Threshold / 2 and float(df.iloc[counter_j,8]) > Threshold / 2 and float(df.iloc[counter_i,8])+float(df.iloc[counter_j,8]) > Threshold:
                        combined_wagging_vibrations[str(i) + "," + str(j)] = float(df.iloc[counter_i,8])+float(df.iloc[counter_j,8])
    
    lst_to_pop= []
    lst_counted = []
    for i in combined_wagging_vibrations.keys():
        if i not in lst_counted:
            i_new = "["+i+"]"
            i_new = json.loads(i_new)
            for j in combined_wagging_vibrations.keys():
                j_new = "["+j+"]"
                j_new = json.loads(j_new)
                if j_new[0] == i_new[-1] and i_new[0] == j_new[-1] and i != lst_counted:
                        lst_to_pop.append(j)
                        lst_counted.append(j)
                    
    for i in lst_to_pop:
        combined_wagging_vibrations.pop(i)
        
    combined_wagging_vibrations.update(single_wagging_vibrations)
    Vibrations["Stretching"] = single_stretching_vibrations
    Vibrations["Bending"] = combined_bending_vibrations
    Vibrations["Wagging"] = combined_wagging_vibrations
    exec("Vibrations_" + str(Molecule) + "[" + str(mode) +"] = Vibrations")

Though it can't be cohesively viewed on Jupyter, we have produced a database that contains all requested information. One way to view that information is by calling a specific part of the dabatase (the value in the parentheses corresponds to the vibrational mode), such as:

In [4]:
Vibrations_vanillin.get(20)

{'Stretching': {},
 'Bending': {},
 'Wagging': {'[2, 1, 6, 14],[7, 1, 6, 14]': 18.799999999999997,
  '[2, 3, 4, 5],[3, 4, 5, 6]': 8.0,
  '[2, 3, 4, 5],[3, 4, 5, 13]': 11.899999999999999,
  '[10, 3, 4, 11],[3, 4, 11, 19]': 5.1,
  '[3, 4, 5, 6],[3, 4, 5, 13]': 11.7,
  '[3, 4, 5, 6],[4, 5, 6, 14]': 15.1,
  '[3, 4, 5, 13],[11, 4, 5, 13]': 20.6,
  '[11, 4, 5, 13],[5, 4, 11, 19]': 15.700000000000001,
  '[3, 4, 11, 19],[5, 4, 11, 19]': 5.0,
  '[2, 1, 6, 14]': 8.7,
  '[7, 1, 6, 14]': 10.1,
  '[2, 3, 4, 5]': 4.1,
  '[3, 4, 5, 13]': 7.8,
  '[11, 4, 5, 13]': 12.8,
  '[4, 5, 6, 14]': 11.2,
  '[13, 5, 6, 1]': 10.7}}